# Handling trajectory data files (reading & writing)

<img align="right" src="https://anitagraser.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/movingpandas/master?filepath=tutorials/2-reading-data-from-files.ipynb)

**<p style="color:#e31883">This notebook demonstrates the current development version of MovingPandas.</p>**

For tutorials using the latest release visit https://github.com/anitagraser/movingpandas-examples.


In [1]:
import pandas as pd
from geopandas import GeoDataFrame, read_file

import sys
sys.path.append("..")
import movingpandas as mpd
mpd.show_versions()

import warnings
warnings.simplefilter("ignore")

ModuleNotFoundError: No module named 'movingpandas'

In [2]:
hvplot_defaults = {'tiles':'EsriImagery', 'frame_height':400, 'frame_width':700, 'cmap':'Viridis', 'colorbar':True}

## Reading Geopackages

### with DatetimeIndex

In [3]:
%%time
gdf = read_file('data/demodata_geolife.gpkg')
gdf['t'] = pd.to_datetime(gdf['t'])
gdf = gdf.set_index('t').tz_localize(None)
traj_collection = mpd.TrajectoryCollection(gdf, 'trajectory_id')
print(traj_collection)

TrajectoryCollection with 5 trajectories
CPU times: user 1.43 s, sys: 21.5 ms, total: 1.46 s
Wall time: 1.57 s


### without DatetimeIndex

In [4]:
%%time
gdf = read_file('data/demodata_geolife.gpkg')
traj_collection = mpd.TrajectoryCollection(gdf, 'trajectory_id', t='t')
print(traj_collection)

TrajectoryCollection with 5 trajectories
CPU times: user 1.45 s, sys: 2.47 ms, total: 1.45 s
Wall time: 1.45 s


In [5]:
%%time
gdf = read_file('data/demodata_ais.gpkg')
gdf['t'] = pd.to_datetime(gdf['Timestamp'], format='%d/%m/%Y %H:%M:%S')
gdf = gdf[gdf.SOG>0]
traj_collection = mpd.TrajectoryCollection(gdf, 'MMSI', min_length=100, t='t')
print(traj_collection)

TrajectoryCollection with 77 trajectories
CPU times: user 10.4 s, sys: 62.4 ms, total: 10.5 s
Wall time: 10.5 s


## Reading CSVs

In [6]:
hvplot_defaults = {'tiles':'EsriImagery', 'frame_height':400, 'frame_width':700, 'cmap':'Viridis', 'colorbar':True}

In [17]:
%%time 
df = pd.read_csv('data/real_bag_0312_1238_pd_2.csv', delimiter=',')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='Y', y='X')
traj_collection.hvplot(title=str(traj_collection), line_width=3, **hvplot_defaults)

CPU times: user 140 ms, sys: 3.8 ms, total: 143 ms
Wall time: 143 ms


:Overlay
   .Tiles.I :Tiles   [x,y]
   .Path.I  :Path   [Longitude,Latitude]

## Reading MovingFeatures JSONs

In [ ]:
%%time 
file_name = 'data/movingfeatures.json'
traj = mpd.read_mf_json(file_name)
traj

In [ ]:
traj.plot()

In [ ]:
traj.hvplot(title='Wind measure along trajectory', c='wind', line_width=5, **hvplot_defaults)

## Writing

### as points

In [ ]:
point_gdf = traj_collection.to_point_gdf()
point_gdf.head()

In [ ]:
point_gdf.to_file("temp.gpkg", layer='points', driver="GPKG")
read_file('temp.gpkg', layer='points').plot()

### as lines

In [ ]:
line_gdf = traj_collection.to_line_gdf()
line_gdf.head()

In [ ]:
line_gdf.to_file("temp.gpkg", layer='lines', driver="GPKG")
read_file('temp.gpkg', layer='lines').plot()

### as trajectories

In [ ]:
traj_gdf = traj_collection.to_traj_gdf(wkt=True)
traj_gdf

In [ ]:
traj_gdf.to_file("temp.gpkg", layer='trajectories', driver="GPKG")
read_file('temp.gpkg', layer='trajectories').plot()

## Error messages while reading

The following errors are expected:

### Missing datetime info

In [ ]:
gdf = read_file('data/demodata_geolife.gpkg')
traj_collection = mpd.TrajectoryCollection(gdf, 'trajectory_id')

### Missing geometry info

In [ ]:
df = pd.read_csv('data/demodata_geolife.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t')

## Continue exploring MovingPandas

1. [Getting started](1-getting-started.ipynb)
1. [Handling trajectory data files (reading & writing)](2-reading-data-from-files.ipynb)
1. [TrajectoryCollection aggregation (flow maps)](3-generalization-and-aggregation.ipynb)
1. [Stop detection](4-stop-detection.ipynb)
1. [Working with local coordinates](5-local-coordinates.ipynb)